<a href="https://colab.research.google.com/github/alberthp/Dijkstra/blob/main/Networks_Dijkstra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Definition of network using JSON
This is an example of the JSON format of the graph representing the network and associated costs

In [ ]:
network_data = {
  "nodes": ["A", "B", "C", "D", "E"],
  "edges": [
    {"source": "A", "destination": "B", "cost": 2},
    {"source": "A", "destination": "C", "cost": 4},
    {"source": "B", "destination": "C", "cost": 1},
    {"source": "B", "destination": "D", "cost": 7},
    {"source": "C", "destination": "E", "cost": 3},
    {"source": "D", "destination": "E", "cost": 1}
  ],
  "start_node": "A"
}

Example 1 of theory slides

In [ ]:
network_data = {
  "nodes": ["U", "V", "W", "X", "Y", "Z"],
  "edges": [
    {"source": "U", "destination": "V", "cost": 2},
    {"source": "U", "destination": "X", "cost": 1},
    {"source": "U", "destination": "W", "cost": 5},
    {"source": "V", "destination": "W", "cost": 3},
    {"source": "V", "destination": "X", "cost": 2},
    {"source": "X", "destination": "W", "cost": 3},
    {"source": "X", "destination": "Y", "cost": 1},
    {"source": "W", "destination": "Y", "cost": 1},
    {"source": "W", "destination": "Z", "cost": 5},
    {"source": "Y", "destination": "Z", "cost": 2}
  ],
  "start_node": "U"
}

Example 2 of theory slides

In [ ]:
network_data = {
  "nodes": ["U", "V", "W", "X", "Y", "Z"],
  "edges": [
    {"source": "U", "destination": "X", "cost": 5},
    {"source": "U", "destination": "W", "cost": 3},
    {"source": "U", "destination": "V", "cost": 7},
    {"source": "X", "destination": "Z", "cost": 9},
    {"source": "X", "destination": "Y", "cost": 7},
    {"source": "X", "destination": "W", "cost": 4},
    {"source": "W", "destination": "Y", "cost": 8},
    {"source": "W", "destination": "V", "cost": 3},
    {"source": "V", "destination": "Y", "cost": 4},
    {"source": "Y", "destination": "Z", "cost": 2}
  ],
  "start_node": "U"
}

# Generation of a network
Run this program to generate a new network if not desired to use a manual graph from previous code.

Steps:
1. Use the slider to define the number of nodes forming the network (between 2 and 10).
2. Press Confirm to generate the network
3. Define the initial node

The program will show the graphical representation of the graph

TODO: Pending to automatically start the computation

In [ ]:
import json
import networkx as nx
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import random

def generate_full_network_interactive():
    """Generates a full network interactively with node selection, confirmation, and edge generation with costs."""
    global network_data_variable
    network_data_variable = None
    nodes = []
    edges_data = []

    num_nodes_widget = widgets.IntSlider(
        value=5,
        min=2,
        max=10,
        step=1,
        description='Number of Nodes:',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='d'
    )
    confirm_nodes_button = widgets.Button(description="Generate Graph")
    network_output = widgets.Output()
    process_dijkstra_button = widgets.Button(description="Process Dijkstra")
    dijkstra_output = widgets.Output()

    display(num_nodes_widget, confirm_nodes_button, network_output, process_dijkstra_button, dijkstra_output)

    def on_confirm_nodes_clicked(b):
        global nodes
        global edges_data
        num_nodes = num_nodes_widget.value
        nodes = [chr(ord('A') + i) for i in range(num_nodes)]
        edges_data = []

        with network_output:
            clear_output()
            if nodes:
                print("Generated Nodes:", nodes)
                G = nx.Graph()
                G.add_nodes_from(nodes)
                possible_edges = []
                for i in range(len(nodes)):
                    for j in range(i + 1, len(nodes)):
                        if random.random() < 0.6:  # Adjust probability for denser/sparser networks
                            cost = random.randint(1, 10)
                            edges_data.append({"source": nodes[i], "destination": nodes[j], "cost": cost})
                            G.add_edge(nodes[i], nodes[j], weight=cost)

                print("\nGenerated Edges:")
                for edge in edges_data:
                    print(f"{edge['source']} --({edge['cost']})-- {edge['destination']}")

                pos = nx.spring_layout(G)
                nx.draw(G, pos, with_labels=True, node_size=1500, node_color="lightblue", font_size=10, font_weight="bold")
                nx.draw_networkx_edge_labels(G, pos, edge_labels={(u, v): d['weight'] for u, v, d in G.edges(data=True)})
                plt.title("Generated Network")
                plt.show()

                # Enable the Dijkstra button and ask for start node
                process_dijkstra_button.disabled = False
                global start_node_widget
                start_node_widget = widgets.Dropdown(
                    options=nodes,
                    value=nodes[0] if nodes else None,
                    description='Start Node:',
                    disabled=not nodes,
                )
                display(start_node_widget)
            else:
                print("No nodes generated.")

    def on_process_dijkstra_clicked(b):
        with dijkstra_output:
            clear_output()
            if nodes and edges_data and hasattr(globals(), 'start_node_widget') and start_node_widget.value:
                start_node = start_node_widget.value
                network_data = {
                    "nodes": nodes,
                    "edges": edges_data,
                    "start_node": start_node
                }
                global network_data_variable
                network_data_variable = network_data
                print("Ready to process Dijkstra's algorithm with the generated network:")
                print(json.dumps(network_data_variable, indent=2))
                print("\nYou can now run the Dijkstra algorithm code in the next cell using the 'network_data_variable'.")
            else:
                print("Please confirm the number of nodes first, and ensure a start node is selected.")

    confirm_nodes_button.on_click(on_confirm_nodes_clicked)
    process_dijkstra_button.on_click(on_process_dijkstra_clicked)
    process_dijkstra_button.disabled = True # Disable until nodes are confirmed

    print("Please use the slider to select the number of nodes and then click 'Confirm Nodes'.")
    return network_data_variable

# Run the interactive network generator
network_data_variable = generate_full_network_interactive()

# Computation of Dijkstra

Program based on Dijktra computation of minimum cost path.

At each step, the program will show the evolution of the graph computation and the table with the values of the algorithm

In [ ]:
# Dijkstra's Algorithm - Step-by-Step Output for Classroom Presentation

import heapq
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import time

# Load network (undirected)
def load_network_from_variable(network_data):
    graph_adj = {node: {} for node in network_data['nodes']}
    edges_list = []
    for edge in network_data['edges']:
        source, destination, cost = edge['source'], edge['destination'], edge['cost']
        graph_adj[source][destination] = cost
        graph_adj[destination][source] = cost
        edges_list.append((source, destination, cost))
        edges_list.append((destination, source, cost))
    return graph_adj, network_data.get('start_node'), edges_list

# Persistent plotting function
def plot_graph(edges, visited_nodes, highlight_edges, graph, current_node=None, step=None):
    G = nx.Graph()
    for u, v, w in edges:
        G.add_edge(u, v, weight=w)
    pos = nx.spring_layout(G, seed=42)

    fig, ax = plt.subplots(figsize=(9, 7))

    # Default edges
    nx.draw_networkx_edges(G, pos, edge_color='gray', ax=ax)

    # Highlighted edges (shortest-path tree)
    if highlight_edges:
        nx.draw_networkx_edges(G, pos, edgelist=highlight_edges, edge_color='red', width=2.5, ax=ax)

    # Highlight current node's edges
    if current_node:
        active_edges = [(current_node, neighbor) for neighbor in graph[current_node] if (current_node, neighbor) in G.edges]
        nx.draw_networkx_edges(G, pos, edgelist=active_edges, edge_color='orange', style='dashed', width=2, ax=ax)

    # Node coloring
    node_colors = []
    for node in G.nodes():
        if node == current_node:
            node_colors.append('orange')
        elif node in visited_nodes:
            node_colors.append('lightgreen')
        else:
            node_colors.append('skyblue')

    nx.draw_networkx_nodes(G, pos, node_color=node_colors, node_size=900, ax=ax)
    nx.draw_networkx_labels(G, pos, font_size=14, font_weight='bold', ax=ax)
    nx.draw_networkx_edge_labels(G, pos, edge_labels={(u, v): d['weight'] for u, v, d in G.edges(data=True)}, ax=ax)

    # Legend
    legend_elements = [
        Patch(facecolor='skyblue', label='Unvisited'),
        Patch(facecolor='lightgreen', label='Visited'),
        Patch(facecolor='orange', label='Current Node'),
        Patch(edgecolor='red', facecolor='white', label='Shortest Path Tree', linewidth=2)
    ]
    ax.legend(handles=legend_elements, loc='lower left')

    ax.set_title(f'Graph State at Step {step}', fontsize=16)
    ax.axis('off')
    plt.show()
import io
import base64
from IPython.display import HTML

def dijkstra_persistent_output(graph, start, edges):
    pq = []
    heapq.heappush(pq, (0, start))
    distances = {node: float('inf') for node in graph}
    distances[start] = 0
    previous_nodes = {node: None for node in graph}
    visited = set()
    table_rows = []
    step = 1

    while pq:
        current_distance, current_node = heapq.heappop(pq)
        if current_node in visited:
            continue

        visited.add(current_node)

        print(f"Step {step}: Visiting node '{current_node}' with distance {current_distance}")

        highlight_edges = [(prev, node) for node, prev in previous_nodes.items() if prev]

        # Update distance table
        row = {'Step': step, "Visited (N')": ', '.join(sorted(visited))}
        for node in graph:
            if distances[node] != float('inf'):
                row[node] = f"{distances[node]} / {previous_nodes[node]}"
            else:
                row[node] = "∞ / None"
        table_rows.append(row)
        table_df = pd.DataFrame(table_rows)

        # Plot graph to buffer
        fig, ax = plt.subplots(figsize=(5, 4))
        G = nx.Graph()
        for u, v, w in edges:
            G.add_edge(u, v, weight=w)
        pos = nx.spring_layout(G, seed=42)

        # Graph edges
        nx.draw_networkx_edges(G, pos, edge_color='gray', ax=ax)
        if highlight_edges:
            nx.draw_networkx_edges(G, pos, edgelist=highlight_edges, edge_color='red', width=2.5, ax=ax)

        # Highlight current node's neighbors
        active_edges = [(current_node, neighbor) for neighbor in graph[current_node] if (current_node, neighbor) in G.edges]
        nx.draw_networkx_edges(G, pos, edgelist=active_edges, edge_color='orange', style='dashed', width=2, ax=ax)

        # Node colors
        node_colors = []
        for node in G.nodes():
            if node == current_node:
                node_colors.append('orange')
            elif node in visited:
                node_colors.append('lightgreen')
            else:
                node_colors.append('skyblue')
        nx.draw_networkx_nodes(G, pos, node_color=node_colors, node_size=900, ax=ax)
        nx.draw_networkx_labels(G, pos, font_size=12, font_weight='bold', ax=ax)
        nx.draw_networkx_edge_labels(G, pos, edge_labels={(u, v): d['weight'] for u, v, d in G.edges(data=True)}, ax=ax)
        ax.axis('off')
        ax.set_title(f"Graph at Step {step}")

        # Save graph image to buffer
        buf = io.BytesIO()
        plt.tight_layout()
        plt.savefig(buf, format='png')
        buf.seek(0)
        graph_base64 = base64.b64encode(buf.getvalue()).decode('utf-8')
        buf.close()
        plt.close(fig)

        # Build HTML display
        html = f"""
        <div style="display: flex; gap: 20px; align-items: flex-start;">
            <div><img src="data:image/png;base64,{graph_base64}" style="border:1px solid #ccc;"/></div>
            <div>{table_df.to_html(index=False)}</div>
        </div>
        """
        display(HTML(html))
        time.sleep(2)

        for neighbor, weight in graph[current_node].items():
            if neighbor not in visited:
                distance = current_distance + weight
                if distance < distances[neighbor]:
                    distances[neighbor] = distance
                    previous_nodes[neighbor] = current_node
                    heapq.heappush(pq, (distance, neighbor))

        step += 1

    return distances, previous_nodes


# Reconstruct path
def reconstruct_path(node, previous_nodes):
    path = []
    while node:
        path.append(node)
        node = previous_nodes[node]
    return path[::-1]

# MAIN
graph, start_node, edges = load_network_from_variable(network_data)
distances, previous_nodes = dijkstra_persistent_output(graph, start_node, edges)

# Final result: Shortest distances and paths
print("\n Final Shortest Distances:")
for node, dist in distances.items():
    print(f"{start_node} -> {node}: {dist}")

print("\n Final Paths:")
for node in graph:
    if node != start_node:
        path = reconstruct_path(node, previous_nodes)
        print(f"Path to {node}: {' -> '.join(path)}")




In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import networkx as nx
from IPython.display import HTML, display

# --- Helper to reconstruct path with costs ---
def reconstruct_path_with_costs(target, previous_nodes, graph):
    path = []
    costs = []
    total_cost = 0
    current = target
    while previous_nodes[current] is not None:
        prev = previous_nodes[current]
        cost = graph[prev][current]
        path.append((prev, current))
        costs.append((f"{prev}→{current}", cost))
        total_cost += cost
        current = prev
    return path[::-1], costs[::-1], total_cost

# --- Animated path visualization with costs ---
def animate_paths_with_costs(graph, edges, previous_nodes, start_node):
    G = nx.Graph()
    for u, v, w in edges:
        G.add_edge(u, v, weight=w)
    pos = nx.spring_layout(G, seed=42)

    all_path_data = []
    for target in graph:
        if target != start_node:
            edge_path, cost_list, total_cost = reconstruct_path_with_costs(target, previous_nodes, graph)
            all_path_data.append((target, edge_path, cost_list, total_cost))

    fig, ax = plt.subplots(figsize=(10, 8))
    ax.axis('off')

    def update(frame):
        ax.clear()
        target, edge_path, cost_list, total_cost = all_path_data[frame]

        # Build readable path
        readable_path = [start_node]
        for u, v in edge_path:
            readable_path.append(v)
        path_str = ' → '.join(readable_path)

        # Build partial cost string
        partial_costs_str = ', '.join([f"{label}: {cost}" for label, cost in cost_list])

        ax.set_title(f"Path to {target}: {path_str}\n"
                     f"Partial Costs: {partial_costs_str}\n"
                     f"Total Cost: {total_cost}", fontsize=13)
        ax.axis('off')

        # Draw full graph
        nx.draw_networkx_edges(G, pos, edge_color='gray', ax=ax)
        nx.draw_networkx_nodes(G, pos, node_color='lightblue', node_size=900, ax=ax)
        nx.draw_networkx_labels(G, pos, font_size=14, font_weight='bold', ax=ax)
        nx.draw_networkx_edge_labels(
            G, pos,
            edge_labels={(u, v): d['weight'] for u, v, d in G.edges(data=True)},
            ax=ax
        )

        # Highlight current path
        nx.draw_networkx_edges(G, pos, edgelist=edge_path, edge_color='red', width=3.0, ax=ax)

    anim = animation.FuncAnimation(fig, update, frames=len(all_path_data), interval=2000, repeat=False)
    plt.close(fig)
    display(HTML(anim.to_jshtml()))

# 🔁 Automatically run the animation after computation
animate_paths_with_costs(graph, edges, previous_nodes, start_node)


